# Data processing

This example walks through the basics for processing data and added metrics.

## Concepts

Devices in the framework contain _raw readings_ that are under the device.readings pandas dataframe. A list of the sensors raw metrics can be shown in device.sensors.

Devices can also contain processed values called metrics. These metrics can be added by passing a callable function and then processed.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scdata as sc
sc._config.config.out_level='DEBUG'
sc._config.config.framework='jupyterlab'

test = sc.Test('EXAMPLE')

In [ ]:
test.load()

## Process basics

In [ ]:
## The readings for each device are accessible via
test.devices['10751'].readings

In [ ]:
## The sensors for each device are accessible via
test.devices['10751'].sensors

In [ ]:
## The metrics for each device are accessible via
test.devices['10751'].metrics

In [ ]:
help(sc.Test.process)

In [ ]:
## Process the metrics as a default
test.process()

## Add metrics

In [ ]:
help(sc.Device.add_metric)

In [ ]:
help(sc.Device.process)

In [ ]:
help(sc.device.process)

In [ ]:
help(sc.device.process.timeseries)
# help(sc.device.process.alphasense)
# help(sc.device.process.regression)

In [ ]:
help(sc.device.process.timeseries.poly_ts)

In [ ]:
metric = {f'TEMP_POLY': {'process': 'poly_ts',
                           'kwargs': {'channels': ['TEMP', 'EXT_TEMP'],
                                      'coefficients': [1, -1]}
                        }}

test.devices['10751'].add_metric(metric)
test.devices['10751'].process(metrics = metric)

In [ ]:
test.devices['10751'].readings

In [ ]:
traces = {1: {'devices': '10751',
              'channel': 'TEMP_POLY',
              'subplot': 1},
          2: {'devices': '10751',
              'channel': 'TEMP',
              'subplot': 1},          
         }

options = {
            'frequency': '1H'
}
test.ts_iplot(traces = traces, options = options);

## Reprocessing

When adding a new metric, one can only process the added metric as above or the whole test (test.process()).

If processes take too long, when adding a metric, the new ones can be processed as: test.reprocess()

In [ ]:
help(sc.Test.reprocess)

In [ ]:
help(sc.device.process.timeseries.clean_ts)

In [ ]:
metric = {f'PM_1_CLEAN': {'process': 'clean_ts',
                           'kwargs': {'name': 'PM_1', 'limits': [0, 1000], 'window_size': 3}
                        }}

test.devices['10751'].add_metric(metric)
test.reprocess()

In [ ]:
traces = {1: {'devices': '10751',
              'channel': 'PM_1',
              'subplot': 1},
          2: {'devices': '10751',
              'channel': 'PM_1_CLEAN',
              'subplot': 1},          
         }

options = {
            'frequency': '1H'
}
test.ts_iplot(traces = traces, options = options);